# **Luke McMeans - Data Project 1**

In [118]:
# imports
import pandas as pd
import json
from dotenv import load_dotenv
import os
import sqlite3
import requests
import time

API Key (hidden in .env file but can be manually inputted)

In [119]:
# get API key
load_dotenv()
api_key = os.getenv("API_KEY", "!! SET DEFAULT HERE")

# **Conversion Functions**

General function to convert a df to a desired file type

In [120]:
def convert_from_df(input_df, result_type):
    # convert based on result type
    output_type = result_type.lower()
    if output_type == "sql":
        conn = sqlite3.connect('output_data/countries.db')
        input_df.to_sql('table_name', conn, if_exists='replace', index=False)
        conn.close()
    elif output_type == "csv":
        input_df.to_csv("output_data/countries.csv", index=False)
    elif output_type == "json":
        input_df.to_json("output_data/countries.json", orient='records')
    else:
        raise ValueError("Desired output isn't SQL, CSV, or JSON")

Handling conversions from SQL file

In [121]:
# converting from sql directly using sqlite
def convert_from_sql(sql_file_path, result_type):    
    # Connect to a temporary in-memory database
    conn = sqlite3.connect(':memory:')
    cursor = conn.cursor()
    
    # read file
    with open(sql_file_path, 'r') as file:
        sql_content = file.read()
    cursor.executescript(sql_content)
    
    # fetch data
    cursor.execute("SELECT * FROM table_name")
    rows = cursor.fetchall()
    
    # get column names
    columns = [description[0] for description in cursor.description]
    
    # create df and convert
    temp_df = pd.DataFrame(rows, columns=columns)
    convert_from_df(temp_df, result_type)
    conn.close()

Handling file conversions for JSON

In [122]:
# converting from json directly
def convert_from_json(json_file_path, result_type):
    # read file
    with open(json_file_path, 'r') as file:
        json_data = json.load(file)
    
    # create df and convert
    temp_df = pd.DataFrame(json_data)
    convert_from_df(temp_df , result_type)


Handling file conversions for CSV

In [123]:
# converting from csv directly
def convert_from_csv(csv_file_path, result_type):
    # read file and convert
    temp_df = pd.read_csv(csv_file_path)
    convert_from_df(temp_df , result_type)

# **Retrieving Input Data (countries_data.csv)**

In [124]:
# load data
df = pd.read_csv('input_data/countries_data.csv')

# show df
df

,name,capital,population,area,region
0,South Georgia,King Edward Point,30,3903.0,Antarctic
1,Grenada,St. George's,112519,344.0,Americas
2,Switzerland,Bern,8654622,41284.0,Europe
3,Sierra Leone,Freetown,7976985,71740.0,Africa
4,Hungary,Budapest,9749763,93028.0,Europe
...,...,...,...,...,...
245,Belgium,Brussels,11555997,30528.0,Europe
246,Israel,Jerusalem,9216900,20770.0,Asia
247,New Zealand,Wellington,5084300,270467.0,Oceania
248,Nicaragua,Managua,6624554,130373.0,Americas


In [125]:
# printing shape
print(f"Number of records: {df.shape[0]}")
print(f"Number of columns: {df.shape[1]}")

# count of countries within each region
region_counts = df['region'].value_counts()
print("\nCount of countries within each region:")
print(region_counts)

# statistical summary of numerical columns
print("\nStatistical summary for 'population' and 'area': ")
df[['population', 'area']].describe().round(2)

Number of records: 250
Number of columns: 5

Count of countries within each region:
region
Africa       59
Americas     56
Europe       53
Asia         50
Oceania      27
Antarctic     5
Name: count, dtype: int64

Statistical summary for 'population' and 'area': 


,population,area
count,2.500000e+02,250.00
mean,3.111089e+07,600584.81
std,1.296673e+08,1909804.91
min,0.000000e+00,0.44
25%,2.210995e+05,1194.25
50%,4.912244e+06,64929.50
75%,1.902577e+07,372726.00
max,1.402112e+09,17098242.00


# **Retrieving API Data**

In [126]:
# list of each country
countries = df['name'].values
countries

array(['South Georgia', 'Grenada', 'Switzerland', 'Sierra Leone',
       'Hungary', 'Taiwan', 'Wallis and Futuna', 'Barbados',
       'Pitcairn Islands', 'Ivory Coast', 'Tunisia', 'Italy', 'Benin',
       'Indonesia', 'Cape Verde', 'Saint Kitts and Nevis', 'Laos',
       'Caribbean Netherlands', 'Uganda', 'Andorra', 'Burundi',
       'South Africa', 'France', 'Libya', 'Mexico', 'Gabon',
       'Northern Mariana Islands', 'North Macedonia', 'China', 'Yemen',
       'Saint Barthélemy', 'Guernsey', 'Solomon Islands',
       'Svalbard and Jan Mayen', 'Faroe Islands', 'Uzbekistan', 'Egypt',
       'Senegal', 'Sri Lanka', 'Palestine', 'Bangladesh', 'Peru',
       'Singapore', 'Turkey', 'Afghanistan', 'Aruba', 'Cook Islands',
       'United Kingdom', 'Zambia', 'Finland', 'Niger', 'Christmas Island',
       'Tokelau', 'Guinea-Bissau', 'Azerbaijan', 'Réunion', 'Djibouti',
       'North Korea', 'Mauritius', 'Montserrat',
       'United States Virgin Islands', 'Colombia', 'Greece', 'Croatia',
   

In [127]:
# create a list to track all data, as well as those countries that couldn't be process
api_data = []
error_countries = []

In [128]:
# retrieve api data for each country
# for country in countries:
#     try:
#         # make api call
#         api_url = 'https://api.api-ninjas.com/v1/country?name={}'.format(country)
#         response = requests.get(api_url, headers={'X-Api-Key': api_key})

#         # add data when retrieved 
#         if response.status_code == 200 and response.json():
#             data = response.json()[0]
#             data['name'] = country
#             api_data.append(data)
#         else:
#             # couldn't get data for the country
#             error_countries.append(country)
#         time.sleep(0.1)
#     except Exception as e:
#         # error thrown when processing country
#         error_countries.append(country)
#         continue

In [129]:
# printing countires that couldn't get data
print("Couldn't retrieve data for the following countries:")
for country in sorted(error_countries):
    print(f"- {country}")

Couldn't retrieve data for the following countries:
- Antarctica
- Bouvet Island
- British Indian Ocean Territory
- Cape Verde
- Caribbean Netherlands
- Christmas Island
- Cocos (Keeling) Islands
- DR Congo
- French Southern and Antarctic Lands
- Guernsey
- Heard Island and McDonald Islands
- Ivory Coast
- Jersey
- Laos
- Macau
- Norfolk Island
- North Korea
- Pitcairn Islands
- Saint Barthélemy
- Saint Martin
- South Georgia
- South Korea
- Svalbard and Jan Mayen
- Taiwan
- United States Minor Outlying Islands
- United States Virgin Islands
- Åland Islands


In [130]:
# turn api data into a df
if not api_data:
    # read from csv to reduce api calls
    api_df = pd.read_csv('api_data.csv')
else:
    api_df = pd.DataFrame(api_data)

# take 'name' variable from currency dict
api_df['currency'] = api_df['currency'].apply(lambda x: x['name'] if isinstance(x, dict) else x)

# bring name to the first column
api_df = api_df[['name'] + [col for col in api_df.columns if col != 'name']]
api_df

,name,gdp,sex_ratio,surface_area,life_expectancy_male,imports,homicide_rate,currency,iso2,urban_population_growth,...,internet_users,gdp_per_capita,fertility,refugees,primary_school_enrollment_male,unemployment,employment_services,employment_industry,employment_agriculture,co2_emissions
0,Grenada,1169.0,101.5,345.0,70.1,470.0,10.8,E. Caribbean Dollar,GD,0.5,...,59.1,10485.9,2.1,0.0,107.7,NaN,NaN,NaN,NaN,NaN
1,Switzerland,705141.0,98.5,41291.0,81.6,276292.0,0.6,Swiss Franc,CH,1.2,...,89.7,82708.5,1.5,120.1,105.5,4.8,77.2,20.0,2.9,37.1
2,Sierra Leone,4101.0,99.6,72300.0,53.2,741.0,1.7,Leone,SL,3.3,...,9.0,536.1,4.3,0.4,110.8,4.4,39.1,6.5,54.4,NaN
3,Hungary,157883.0,90.8,93024.0,73.0,116556.0,2.5,Forint,HU,0.2,...,76.1,16264.0,1.5,6.4,101.1,3.5,62.6,32.8,4.6,45.8
4,Wallis and Futuna,NaN,93.4,142.0,72.8,65.0,NaN,Cfp Franc,WF,0.0,...,9.0,NaN,2.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
218,Belgium,543026.0,98.3,30528.0,79.0,426489.0,1.7,Euro,BE,0.7,...,88.7,47293.0,1.7,83.5,103.9,5.7,78.4,20.6,0.9,90.4
219,Israel,370588.0,99.1,22072.0,81.0,76579.0,1.5,New Israeli Sheqel,IL,1.7,...,83.7,44214.9,3.0,55.1,104.4,3.8,82.3,16.8,0.9,63.8
220,New Zealand,207921.0,96.7,268107.0,80.3,42271.0,0.7,New Zealand Dollar,NZ,1.1,...,90.8,43836.2,1.9,2.6,99.8,4.0,75.1,19.4,5.6,32.2
221,Nicaragua,13118.0,97.2,130373.0,70.6,8426.0,7.2,Cordoba Oro,NI,1.5,...,27.9,2028.9,2.4,0.8,121.1,7.4,53.8,15.6,30.6,5.1


In [131]:
# saving to directory to reduce api usage
api_df.to_csv('api_data.csv', index=False)

In [132]:
# printing shape
print(f"Number of records: {api_df.shape[0]}")
print(f"Number of columns: {api_df.shape[1]}")

# count of countries within each region
region_counts = api_df['region'].value_counts()
print("\nCount of countries within each region:")
print(region_counts)

# statistical summary of some numerical columns
print("\nStatistical summary for specific columns: ")
api_df[['gdp', 'gdp_per_capita', 'urban_population_growth', 'life_expectancy_male', 'life_expectancy_female', 'internet_users', 'imports']].describe().round(2)

Number of records: 223
Number of columns: 38

Count of countries within each region:
region
Caribbean             21
Eastern Africa        19
Western Asia          18
Southern Europe       17
Western Africa        15
South America         14
Western Europe        13
Northern Europe       12
Eastern Europe        10
South-Eastern Asia    10
Polynesia              9
Southern Asia          9
Middle Africa          8
Central America        8
Micronesia             7
Northern Africa        7
Melanesia              5
Central Asia           5
Southern Africa        5
Northern America       5
Eastern Asia           4
Oceania                2
Name: count, dtype: int64

Statistical summary for specific columns: 


,gdp,gdp_per_capita,urban_population_growth,life_expectancy_male,life_expectancy_female,internet_users,imports
count,206.00,206.00,223.00,209.00,209.00,210.00,209.00
mean,441817.87,18298.45,1.86,70.46,75.38,57.59,98406.64
std,1817674.72,26982.59,1.76,7.33,7.77,28.53,277064.04
min,46.00,99.60,-3.60,50.40,54.80,1.30,1.00
25%,6810.75,2276.22,0.50,66.00,71.30,33.65,1711.00
50%,30422.50,7086.70,1.70,71.30,77.50,64.15,8370.00
75%,235223.25,23160.92,3.05,76.20,80.80,80.85,52696.00
max,20580223.00,185835.00,8.00,81.80,87.50,99.70,2567490.00


# **Manipulating and Merging Data**

In [133]:
# print all columns from api dataframe
print(api_df.columns)

Index(['name', 'gdp', 'sex_ratio', 'surface_area', 'life_expectancy_male',
       'imports', 'homicide_rate', 'currency', 'iso2',
       'urban_population_growth', 'secondary_school_enrollment_female',
       'capital', 'forested_area', 'tourists', 'exports',
       'life_expectancy_female', 'post_secondary_enrollment_female',
       'post_secondary_enrollment_male', 'primary_school_enrollment_female',
       'infant_mortality', 'gdp_growth', 'threatened_species', 'population',
       'urban_population', 'secondary_school_enrollment_male', 'pop_growth',
       'region', 'pop_density', 'internet_users', 'gdp_per_capita',
       'fertility', 'refugees', 'primary_school_enrollment_male',
       'unemployment', 'employment_services', 'employment_industry',
       'employment_agriculture', 'co2_emissions'],
      dtype='object')


In [134]:
# calculate average life expectancy using sex ratio and life expectancy of males/females
api_df['average_life_expectancy'] = api_df.apply(lambda row: 
    (row['life_expectancy_male'] * row['sex_ratio'] + row['life_expectancy_female'] * 100) / (row['sex_ratio'] + 100)
    if pd.notnull(row['life_expectancy_male']) and pd.notnull(row['life_expectancy_female']) and pd.notnull(row['sex_ratio'])
    else None, axis=1)

# display new data
display(api_df[['name', 'sex_ratio', 'life_expectancy_male', 'life_expectancy_female', 'average_life_expectancy']])

,name,sex_ratio,life_expectancy_male,life_expectancy_female,average_life_expectancy
0,Grenada,101.5,70.1,75.0,72.531762
1,Switzerland,98.5,81.6,85.4,83.514358
2,Sierra Leone,99.6,53.2,54.8,54.001603
3,Hungary,90.8,73.0,80.1,76.721174
4,Wallis and Futuna,93.4,72.8,78.7,75.850672
...,...,...,...,...,...
218,Belgium,98.3,79.0,83.7,81.370146
219,Israel,99.1,81.0,84.3,82.657459
220,New Zealand,96.7,80.3,83.8,82.079359
221,Nicaragua,97.2,70.6,77.7,74.200406


**Combining DataFrames**

Given the API is 'live' as opposed to a preloaded csv file, we will use the API's population data. It also can tie into other population-related statistics, so that's a plus. However, I will be using the CSV's region column as it provides a smaller, cleaner set of categorical variables to analyze if needed. Area will also be used from the CSV.

Here's the columns we'll pull from the two DFs:
- Name
- Capital 
- Region
- Area
- Population
- Population Density
- Urban Population
- Population Growth
- Urban Population Growth
- GDP
- GDP Per Capita
- GDP Growth
- Life Expectancy (Average, Male, and Female)
- Unemployment
- Internet Users

In [135]:
# merge the two dataframes together
# only include countries in both dfs
df_1 = df[~df['name'].isin(error_countries)][['name', 'capital', 'region','area']].sort_values(by='name')
df_2 = api_df[['name', 'population', 'pop_density', 'urban_population', 'pop_growth',
               'urban_population_growth', 'gdp', 'gdp_per_capita', 'gdp_growth', 
               'average_life_expectancy', 'life_expectancy_male', 'life_expectancy_female',
               'unemployment', 'internet_users']].sort_values(by='name')
merged_df = pd.merge(df_1, df_2, on='name')

merged_df

,name,capital,region,area,population,pop_density,urban_population,pop_growth,urban_population_growth,gdp,gdp_per_capita,gdp_growth,average_life_expectancy,life_expectancy_male,life_expectancy_female,unemployment,internet_users
0,Afghanistan,Kabul,Asia,652230.0,38928.0,59.6,25.8,2.5,4.0,20514.0,551.9,-1.7,64.260565,62.8,65.8,11.2,13.5
1,Albania,Tirana,Europe,28748.0,2878.0,105.0,61.2,-0.1,1.8,15059.0,5223.8,4.1,78.369121,76.7,80.1,12.8,71.8
2,Algeria,Algiers,Africa,2381741.0,43851.0,18.4,73.2,2.0,2.9,173757.0,4114.7,1.4,76.587531,75.4,77.8,11.5,49.0
3,American Samoa,Pago Pago,Oceania,199.0,56.0,279.0,87.1,0.1,-0.1,NaN,NaN,NaN,74.390766,71.1,77.8,9.2,NaN
4,Andorra,Andorra la Vella,Europe,468.0,77.0,164.2,88.0,-0.2,-1.7,3238.0,42051.6,1.6,NaN,NaN,NaN,NaN,91.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
218,Wallis and Futuna,Mata-Utu,Oceania,142.0,12.0,82.5,0.0,-0.9,0.0,NaN,NaN,NaN,75.850672,72.8,78.7,NaN,9.0
219,Western Sahara,El Aaiún,Africa,266000.0,597.0,2.2,86.8,2.5,1.9,NaN,NaN,NaN,69.966110,68.2,71.9,9.4,NaN
220,Yemen,Sana'a,Asia,527968.0,29826.0,56.5,37.3,2.4,4.4,26672.0,935.9,-1.3,66.037717,64.4,67.7,12.8,26.7
221,Zambia,Lusaka,Africa,752612.0,18384.0,24.7,44.1,2.9,4.3,27283.0,1572.3,4.1,63.278294,60.3,66.2,11.4,14.3
